In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
from sklearn.metrics import *

In [ ]:
def evaluate(val_ds, model, device, tokenizer):
    # 입력 데이터셋 토크나이징 (attention_mask 포함)
    inputs = tokenizer(val_ds['text'], return_tensors="pt", padding=True,
                       truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 입력 텐서를 동일한 디바이스로 이동

    model.eval()
    model = model.to(device)

    with torch.no_grad():
        outputs = model(**inputs)  # attention_mask를 포함해 입력

    # 다중 라벨 분류를 위한 sigmoid 함수 적용
    probabilities = torch.sigmoid(outputs.logits)

    if probabilities.is_cuda:
        probabilities = probabilities.cpu().detach().numpy()
    else:
        probabilities = probabilities.detach().numpy()

    # 확률에 대해 threshold 적용하여 다중 라벨 예측 (0 또는 1)
    threshold = 0.5
    y_pred = (probabilities > threshold).astype(int)

    # 최소 1개 이상 클래스 예측 보장 (argmax 적용)
    for i in range(y_pred.shape[0]):
        if y_pred[i].sum() == 0:  # 만약 선택된 클래스가 없으면
            max_idx = np.argmax(probabilities[i])  # 가장 높은 확률을 가진 클래스를 선택
            y_pred[i, max_idx] = 1  # 강제 예측

    del inputs
    torch.cuda.empty_cache()

    return y_pred, probabilities

In [ ]:
def predict(text, model, tokenizer, device='cpu'):
    # 입력 문장 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 각 텐서를 GPU로 이동

    model = model.to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.sigmoid(logits)  # Sigmoid로 확률 계산
    probabilities = probabilities.squeeze()  # 단일 예측에 대해 차원 축소
    threshold = 0.5
    y_pred = (probabilities > threshold).astype(int)

    # 최소 1개 이상 클래스 예측 보장 (argmax 적용)
    if y_pred.sum() == 0:  # 만약 선택된 클래스가 없으면
        max_idx = np.argmax(probabilities)  # 가장 높은 확률을 가진 클래스를 선택
        y_pred[max_idx] = 1  # 강제 예측

    return y_pred, probabilities

In [ ]:
data = pd.read_csv('20250204_기존데이터추가_최종완료.csv')
print(data.head())

                                     text label
0              스님들 사람들 안보이는데서는 고기 먹는거 아냐?   [0]
1         요즘 계속 체한것 때문에 고생이래 음식을 먹을 수 있을까   [0]
2  땀 구멍이 좀 크네요.. 눈이 작으니 아이라인을 크게 그려야 겠어요.   [0]
3          확실히 광주보다는 대구가 훠얼씬 살기 좋은거 같아 ㅋㅋ   [0]
4                     내가 머리 기르면 여자처럼 보일까.   [0]


In [ ]:
data.shape

(18113, 2)

In [ ]:
train, val = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# 텐서 데이터셋으로 변환
train_ds = Dataset.from_pandas(train)
val_ds = Dataset.from_pandas(val)

In [ ]:
# 모델과 토크나이저 불러오기
model_name = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

class CustomDataCollator:
    def __call__(self, batch):
        # 이 부분에서, item['input_ids']는 이미 torch.Tensor 형태
        input_ids = [item['input_ids'] for item in batch]
        attention_mask = [item['attention_mask'] for item in batch]
        labels = [item['labels'] for item in batch]

        return {
            'input_ids': pad_sequence(input_ids, batch_first=True, padding_value=0),
            'attention_mask': pad_sequence(attention_mask, batch_first=True, padding_value=0),
            'labels': torch.stack(labels),
        }

In [ ]:
import ast

# 토큰화 함수 생성 및 적용
def preprocess_function(data):
    # 텍스트 토큰화
    inputs = tokenizer(data['text'], truncation=True, padding=True, max_length=512)

    num_labels = 12  # 클래스 수
    # 'label'을 원-핫 인코딩
    one_hot_labels = []
    for label in data['label']:
        # label이 문자열로 저장된 리스트라면 이를 파싱
        parsed_labels = ast.literal_eval(label) if isinstance(label, str) else label
        one_hot = [0] * num_labels  # 0으로 초기화된 원-핫 벡터
        for l in parsed_labels:
            one_hot[int(l)] = 1
        one_hot_labels.append(one_hot)

    # 원-핫 인코딩된 레이블을 텐서로 변환
    inputs['labels'] = torch.tensor(one_hot_labels, dtype=torch.float32)
    return inputs

train_ds = train_ds.map(preprocess_function, batched=True)
val_ds = val_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/14490 [00:00<?, ? examples/s]

Map:   0%|          | 0/3623 [00:00<?, ? examples/s]

In [ ]:
# 텐서로 변환
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# 모델 설정 (다중 분류를 위해 num_labels 지정)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 12, problem_type="multi_label_classification").to(device)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# TrainingArguments
training_args = TrainingArguments(
    output_dir = './results',          # 출력 디렉토리
    eval_strategy = "epoch",            # 에폭마다 평가
    save_strategy = "epoch",           # 에폭마다 체크포인트 저장
    learning_rate = 1e-5,              # <--[조정가능]학습률
    per_device_train_batch_size = 32,  # <--[조정가능]학습 배치 사이즈
    per_device_eval_batch_size = 32,   # <--[조정가능]평가 배치 사이즈
    num_train_epochs  = 30,              # <--[조정가능]에폭 수
    weight_decay = 0.01,               # <--[조정가능]weight decay
    load_best_model_at_end = True,     # 가장 좋은 모델을 마지막에 로드
    logging_dir ='./logs',            # 로깅 디렉토리
    logging_steps = 10,                # 로깅 스텝
    report_to="tensorboard",           # TensorBoard에 로깅
    fp16=True
)

In [ ]:
# Trainer 설정
data_collator = CustomDataCollator()

trainer = Trainer(
    model=model,                         # 학습할 모델
    args=training_args,                  # TrainingArguments
    train_dataset = train_ds,
    eval_dataset = val_ds,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],# 조기 종료
    data_collator=data_collator,
)

<ipython-input-16-48f1bea9daba>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.178300,0.174624
2,0.124700,0.119063
3,0.081700,0.093682
4,0.065400,0.094605
5,0.060900,0.084649
6,0.045200,0.085017
7,0.038100,0.084081
8,0.043300,0.084478
9,0.034800,0.085600
10,0.027500,0.084684


TrainOutput(global_step=4530, training_loss=0.08148313089711777, metrics={'train_runtime': 2286.3229, 'train_samples_per_second': 190.131, 'train_steps_per_second': 5.944, 'total_flos': 3.8128214992896e+16, 'train_loss': 0.08148313089711777, 'epoch': 10.0})

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 모델과 토크나이저 저장 경로
output_dir = "./fine_tuned_model"

# 모델 저장
trainer.save_model(output_dir)

# 토크나이저 저장 (토크나이저가 필요하면 함께 저장)
tokenizer.save_pretrained(output_dir)

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
# 모델 평가
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.08408121019601822, 'eval_runtime': 15.3438, 'eval_samples_per_second': 236.121, 'eval_steps_per_second': 7.43, 'epoch': 10.0}


In [ ]:
pred, prob = evaluate(val_ds, model, device, tokenizer)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# 다중 레이블을 리스트로 변환
val_df = val_ds.to_pandas()
val_df['label'] = val_df['label'].apply(lambda x: list(map(int, x.strip("[]").split(","))))

# 멀티라벨 인코딩
mlb = MultiLabelBinarizer()
y_true = mlb.fit_transform(val_df['label'])

y_pred = pred

# 평가
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=[str(cls) for cls in mlb.classes_]))


              precision    recall  f1-score   support

           0       0.94      0.95      0.94      1235
           1       0.97      0.97      0.97      2388
           2       0.73      0.69      0.71       276
           3       0.88      0.80      0.84       245
           4       0.90      0.88      0.89       216
           5       0.82      0.82      0.82       290
           6       0.87      0.93      0.90       211
           7       0.91      0.89      0.90       201
           8       0.90      0.83      0.86       223
           9       0.90      0.83      0.87       201
          10       0.67      0.91      0.77       187
          11       0.80      0.68      0.74       243

   micro avg       0.91      0.91      0.91      5916
   macro avg       0.86      0.85      0.85      5916
weighted avg       0.91      0.91      0.91      5916
 samples avg       0.92      0.92      0.91      5916



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def split_text(text, max_length, tokenizer):
    """텍스트를 토큰 길이를 기준으로 나눔"""
    tokens = tokenizer(text, truncation=False, padding=False)
    input_ids = tokens["input_ids"]
    chunks = [input_ids[i:i + max_length] for i in range(0, len(input_ids), max_length)]
    # 토큰 ID를 다시 텍스트로 디코딩
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks if len(chunk) > 0]


# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")

# 입력 텍스트
input_text = """
뭔가 틀딱 알고리즘맞는거같노... 시발 홍어새끼들 틀딱새끼들 죄다 잡아서 바이든 후장에 넣으면 해결되는거냐?
"""

# 텍스트 조각 나누기
max_length = 512
chunks = split_text(input_text, max_length - 2, tokenizer)  # [CLS]와 [SEP]을 고려하여 max_length - 2

# 각 조각에 대해 예측
results = []
model.eval()
for chunk in chunks:
    # 각 텍스트 조각을 토큰화
    try:
        inputs = tokenizer(chunk, truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.sigmoid(logits).squeeze()  # Sigmoid로 확률 계산
            results.append(probabilities)
    except Exception as e:
        print(f"Error processing chunk: {chunk}")
        print(f"Error details: {e}")

# 결과 병합
if results:
    final_probabilities = torch.mean(torch.stack(results), dim=0)  # 평균 확률
    threshold = 0.5
    predicted_labels = (final_probabilities > threshold).nonzero(as_tuple=True)[0].tolist()

    # 라벨 매핑
    label_mapping = {
        0: "정상",
        1: "악성",
        2: "욕설",
        3: "외모",
        4: "장애인",
        5: "인종",
        6: "종교",
        7: "지역",
        8: "성차별",
        9: "나이",
        10: "협박",
        11: "성희롱",
    }

    # 클래스별 확률 출력 (디버깅용)
    print("클래스별 확률:")
    for i, prob in enumerate(final_probabilities.tolist()):
        print(f"{label_mapping[i]}: {prob:.4f}")

    # 결과 출력
    predicted_labels_text = [label_mapping[label] for label in predicted_labels]
    print(f"입력 텍스트: {input_text}")
    print(f"예측 라벨: {predicted_labels_text}")
else:
    print("텍스트 조각 처리 중 문제가 발생하여 결과를 생성할 수 없습니다.")


클래스별 확률:
정상: 0.0196
악성: 0.9827
욕설: 0.2832
외모: 0.0198
장애인: 0.0257
인종: 0.0677
종교: 0.0686
지역: 0.9780
성차별: 0.0565
나이: 0.8449
협박: 0.0307
성희롱: 0.0477
입력 텍스트: 
뭔가 틀딱 알고리즘맞는거같노... 시발 홍어새끼들 틀딱새끼들 죄다 잡아서 바이든 후장에 넣으면 해결되는거냐?

예측 라벨: ['악성', '지역', '나이']


In [ ]:
def split_text(text, max_length, tokenizer):
    """텍스트를 토큰 길이를 기준으로 나눔"""
    tokens = tokenizer(text, truncation=False, padding=False)
    input_ids = tokens["input_ids"]
    chunks = [input_ids[i:i + max_length] for i in range(0, len(input_ids), max_length)]
    # 토큰 ID를 다시 텍스트로 디코딩
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks if len(chunk) > 0]

# 모델과 토크나이저 로드 (순화 전 예측)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")

# 입력 텍스트
input_text = """
여기는 뭐하는 곳입니까?
길거리에 멧돼지가 돌아다니고 있는데
"""

# 텍스트 조각 나누기 ([CLS]와 [SEP]를 고려하여 max_length - 2)
max_length = 512
chunks = split_text(input_text, max_length - 2, tokenizer)

results = []
model.eval()
for chunk in chunks:
    try:
        inputs = tokenizer(chunk, truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.sigmoid(logits).squeeze()  # Sigmoid로 확률 계산
            results.append(probabilities)
    except Exception as e:
        print(f"Error processing chunk: {chunk}")
        print(f"Error details: {e}")

# 결과 병합 및 예측
if results:
    final_probabilities = torch.mean(torch.stack(results), dim=0)  # 평균 확률
    threshold = 0.5
    predicted_labels = (final_probabilities > threshold).nonzero(as_tuple=True)[0].tolist()

    # 라벨 매핑
    label_mapping = {
        0: "정상",
        1: "악성",
        2: "욕설",
        3: "외모",
        4: "장애인",
        5: "인종",
        6: "종교",
        7: "지역",
        8: "성차별",
        9: "나이",
        10: "협박",
        11: "성희롱",
    }

    # 클래스별 확률 출력 (디버깅용)
    print("클래스별 확률:")
    for i, prob in enumerate(final_probabilities.tolist()):
        print(f"{label_mapping[i]}: {prob:.4f}")

    # 결과 출력
    predicted_labels_text = [label_mapping[label] for label in predicted_labels]
    print(f"입력 텍스트: {input_text}")
    print(f"예측 라벨: {predicted_labels_text}")
else:
    print("텍스트 조각 처리 중 문제가 발생하여 결과를 생성할 수 없습니다.")

클래스별 확률:
정상: 0.9938
악성: 0.0057
욕설: 0.0039
외모: 0.0032
장애인: 0.0042
인종: 0.0041
종교: 0.0029
지역: 0.0046
성차별: 0.0030
나이: 0.0033
협박: 0.0042
성희롱: 0.0038
입력 텍스트: 
여기는 뭐하는 곳입니까?
길거리에 멧돼지가 돌아다니고 있는데

예측 라벨: ['정상']


In [ ]:
# 오버랩 구현

def split_text(text, max_length, tokenizer, overlap=50):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    total_len = len(tokens)

    if max_length <= 0:
        raise ValueError("max_length must be a positive integer.")
    if overlap >= max_length:
        raise ValueError("overlap은 max_length보다 작아야 합니다.")

    step = max_length - overlap
    chunks = []
    start = 0
    while start < total_len:
        end = start + max_length
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        if chunk_text.strip():
            chunks.append(chunk_text)
        start += step

    return chunks

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")

# 입력 텍스트 (장문 가능)
input_text = """
안녕하세요
봉천동에 살고 있고 신림에 직장이 있는 청년입니다
평소엔 걷거나 따릉이를 타고 다니는데 오늘 눈이 와서 오랜만에 지하철을 이용했습니다.
눈이 온 신림역 1번 출구 언덕은 지옥이 따로 없었습니다.
잡을 수 있는 안전봉도 없고 나이든 어르신 뿐만 아니라 젊은 사람들도 넘어질까 겁이 나서 다들 주춤 거리며 내려오고 있었습니다.
물론 제가 생각하는것 만큼 간단한 문제는 아니겠으나 비나 눈이 왔을때 주민들이 안전하게 내려올수 있도록 계단을 설치해주시거나 길 양쪽에 안전봉이라도 설치해주셨으면 합니다.
신림역부터 양지병원까지 가는 길이 너무 위험해 오히려 환자를 만드는건 아닌가 하는 생각이 들었습니다.
지금도 너무 수고하고 계시나 아이부터 어른까지 조금 더 안전한 관악구가 될수 있도록 부탁드립니다 감사합니다
"""

# 텍스트 조각 나누기
max_length = 384
overlap = 50
chunks = split_text(input_text, max_length=max_length, tokenizer=tokenizer, overlap=overlap)

results = []
model.eval()
for chunk in chunks:
    try:
        inputs = tokenizer(
            chunk,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.sigmoid(logits).squeeze()
            results.append(probabilities)
    except Exception as e:
        print(f"Error processing chunk: {chunk}")
        print(f"Error details: {e}")

# 결과 병합
if results:
    final_probabilities = torch.mean(torch.stack(results), dim=0)
    threshold = 0.5
    predicted_labels = (final_probabilities > threshold).nonzero(as_tuple=True)[0].tolist()

    label_mapping = {
        0: "정상",
        1: "악성",
        2: "욕설",
        3: "외모",
        4: "장애인",
        5: "인종",
        6: "종교",
        7: "지역",
        8: "성차별",
        9: "나이",
        10: "협박",
        11: "성희롱",
    }

    print("클래스별 확률:")
    for i, prob in enumerate(final_probabilities.tolist()):
        print(f"{label_mapping[i]}: {prob:.4f}")

    predicted_labels_text = [label_mapping[label] for label in predicted_labels]
    print(f"입력 텍스트: {input_text}")
    print(f"예측 라벨: {predicted_labels_text}")
else:
    print("텍스트 조각 처리 중 문제가 발생하여 결과를 생성할 수 없습니다.")


클래스별 확률:
정상: 0.9940
악성: 0.0056
욕설: 0.0043
외모: 0.0046
장애인: 0.0047
인종: 0.0036
종교: 0.0031
지역: 0.0052
성차별: 0.0036
나이: 0.0045
협박: 0.0035
성희롱: 0.0037
입력 텍스트: 
안녕하세요
봉천동에 살고 있고 신림에 직장이 있는 청년입니다
평소엔 걷거나 따릉이를 타고 다니는데 오늘 눈이 와서 오랜만에 지하철을 이용했습니다.
눈이 온 신림역 1번 출구 언덕은 지옥이 따로 없었습니다.
잡을 수 있는 안전봉도 없고 나이든 어르신 뿐만 아니라 젊은 사람들도 넘어질까 겁이 나서 다들 주춤 거리며 내려오고 있었습니다.
물론 제가 생각하는것 만큼 간단한 문제는 아니겠으나 비나 눈이 왔을때 주민들이 안전하게 내려올수 있도록 계단을 설치해주시거나 길 양쪽에 안전봉이라도 설치해주셨으면 합니다.
신림역부터 양지병원까지 가는 길이 너무 위험해 오히려 환자를 만드는건 아닌가 하는 생각이 들었습니다.
지금도 너무 수고하고 계시나 아이부터 어른까지 조금 더 안전한 관악구가 될수 있도록 부탁드립니다 감사합니다

예측 라벨: ['정상']


**기본 모델 성능**

In [ ]:
# Hugging Face 라이브러리에서 사전 학습된 모델 로드
base_model = AutoModelForSequenceClassification.from_pretrained(
    "klue/roberta-base", num_labels=12, problem_type="multi_label_classification"
).to(device)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 사전 학습된 모델을 사용하여 예측
base_pred, base_prob = evaluate(val_ds, base_model, device, tokenizer)


In [ ]:
import ast

# 데이터프레임 변환
val_df = val_ds.to_pandas()

# 만약 'label'이 문자열로 저장되어 있다면 리스트로 변환
val_df["label"] = val_df["label"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# MultiLabelBinarizer 적용
mlb = MultiLabelBinarizer()
y_true = mlb.fit_transform(val_df["label"])  # 정답 데이터 변환


In [ ]:
# 기본 모델의 예측값 (evaluate() 함수 호출)
base_pred, base_prob = evaluate(val_ds, base_model, device, tokenizer)

# 예측값도 동일한 클래스 개수를 가지도록 변환
base_pred = mlb.transform(base_pred)


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# 성능 평가
base_report = classification_report(y_true, base_pred, target_names=[str(cls) for cls in mlb.classes_])

# 결과 출력
print("🔹 기본 KLUE-RoBERTa 모델 성능 평가 결과:")
print(base_report)


🔹 기본 KLUE-RoBERTa 모델 성능 평가 결과:
              precision    recall  f1-score   support

           0       0.34      1.00      0.51      1235
           1       0.66      1.00      0.79      2388
           2       0.00      0.00      0.00       276
           3       0.00      0.00      0.00       245
           4       0.00      0.00      0.00       216
           5       0.00      0.00      0.00       290
           6       0.00      0.00      0.00       211
           7       0.00      0.00      0.00       201
           8       0.00      0.00      0.00       223
           9       0.00      0.00      0.00       201
          10       0.00      0.00      0.00       187
          11       0.00      0.00      0.00       243

   micro avg       0.50      0.61      0.55      5916
   macro avg       0.08      0.17      0.11      5916
weighted avg       0.34      0.61      0.43      5916
 samples avg       0.50      0.70      0.56      5916



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
